In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.stats import pearsonrimport numpy as np


In [134]:
def sk_linreg(df, coef=False):
    df.dropna(inplace=True)
    df_linreg = pd.DataFrame()
    for region in np.unique(df['Region']):
        df_ = df[df['Region']==region].copy()
        df_.dropna(inplace=True)
        X = np.array(df_['Year']).reshape(-1, 1)
        reg = LinearRegression().fit(X, df_.iloc[:,-1])
        df_['Linear Regression Line'] = reg.predict(X.reshape(-1, 1))[0]
        df_linreg = pd.concat([df_linreg, df_])
    print(coef)
    if coef:
        return df_linreg, reg.coef_
    return df_linreg

def plot(df_, region=None):
    if region == 'Davos':
        df_ = df_[df_['Region']=='Davos'].copy()
        df_.dropna(inplace=True)
    data_name = df_.columns[-1]
    fig = go.Figure()
    for region in np.unique(df_['Region']):
        df = df_[df_['Region']==region]
        fig.add_trace(go.Scatter(x=df['Year'], y=df[data_name],
                                 name=region, line=dict(color='red')
                                 ))

    fig.update_layout(title=data_name + ' pro Jahr' + ' in Davos' if region=='Davos' else '',
                   xaxis_title='Jahr',
                    template='simple_white',
                   yaxis_title=data_name + ' [cm]')
    return fig

def plot_linreg(df, data_name='Neuschnee'):
    df.dropna(inplace=True)
    df, coef = sk_linreg(df, True)
    fig = go.Figure()
    unit = '[cm]' if data_name=='Neuschnee' else ''
    fig.add_trace(go.Scatter(x=df['Year'], y=df[data_name], name=data_name, line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df['Year'], y=df['Linear Regression Line'], name='Linear Regression'))
    fig.update_layout(title='{} pro Jahr. Veränderung von {:.2} {} pro Jahr'.format(data_name, coef[0], unit),
                      xaxis_title='Jahr',
                      showlegend=False,
                      template='simple_white',
                      yaxis_title=data_name + ' ' + unit)
    return fig

def plot_linreg_subplots(df, data_name='Neuschnee'):
    coefs = []
    for region in np.unique(df['Region']):
        df_ = df[df['Region']==region].copy()
        df_, coef = sk_linreg(df_, True)
        coefs.append(coef[0])
    
    subplots_titles = []
    for coef, region in zip(coefs, np.unique(df['Region'])):
        subplots_titles.append('{} in {}. Veränderung von {:.2} pro Jahr.'.format(data_name, region, coef))
    
    nr_region = len(np.unique(df['Region']))
    fig = make_subplots(rows=nr_region, cols=1, subplot_titles=subplots_titles)
    
    for i, region in enumerate(np.unique(df['Region']), 1):

        df_ = df[df['Region']==region].copy()
        df_, coef = sk_linreg(df_, True)
        fig.append_trace(go.Scatter(x=df_['Year'], y=df_[data_name],
                                    name=data_name),
                         row = i, col = 1)
        fig.append_trace(go.Scatter(x=df_['Year'], y=df_['Linear Regression Line'], name=np.round(coef[0],2)),
                         row = i, col = 1)
    
    
    fig.update_layout(height=3000, width=600, title_text=data_name+ " und Lineare Regression.",
                      showlegend=False, xaxis_title='Jahr', template='simple_white',
                      yaxis_title=data_name)
    return fig


def calculate_corr(df1, df2):
    df1.dropna(inplace=True)
    df2.dropna(inplace=True)
    corrs = []
    regions = []

    for region in np.unique(df1['Region']):
        df1_ = df1[df1['Region']==region].copy()
        df2_ = df2[df2['Region']==region].copy()
        first_year = max(df1_['Year'].min(), df2_['Year'].min())
        df1_ = df1_[df1_['Year']>=first_year].copy()
        df2_ = df2_[df2_['Year']>=first_year].copy()
        df1_.sort_values(by='Year', inplace=True)
        df2_.sort_values(by='Year', inplace=True)
        try:
            corr, _ = pearsonr(df1_.iloc[:,-1], df2_.iloc[:,-1])
            corrs.append(corr)
            regions.append(region)
        except Exception as E:
            print(E, "with ", region)

    corr_df = pd.DataFrame(list(zip(regions,corrs)), columns=['Region', 'Correlation'])
    return corr_df


def lineplot_swiss_temp(data, data_name = 'Neuschnee'):
    fig = go.Figure()
    for region in np.unique(data['Region']):
        df_ = data[data['Region']==region].copy()
        df_[data_name] = df_[data_name] - df_[data_name].mean()
        if region != 'Davos':
            fig.add_trace(go.Scatter(x=df_['Year'],
                                     y=df_[data_name],
                                     name=data_name,
                                     line=dict(color='grey'),
                                     opacity=0.3)
                         )
        else:
            fig.add_trace(go.Scatter(x=df_['Year'],
                                     y=df_[data_name],
                                     name=data_name,
                                     line=dict(color='red',
                                               width=2)
                                    )
                         )
    fig.add_trace(go.Scatter(x=data['Year'],
                             y=data[data_name]*0,
                             name='Zentriertes Mittel',
                             line=dict(color='black',
                                       dash='dot',
                                       width=1),
                             opacity=0.8)
                 )
    fig.update_layout(template='simple_white',
                      title='Temperaturabweichungen aller Orte zu ihrem Mittelwert',
                      showlegend=False,
                      yaxis_title='Temperaturdifferenz [K]',
                      xaxis_title='Jahre')
    fig.show()


def selectable_region_plot(df, data_name = 'Neuschnee'):
    df = sk_linreg(df, False).copy()
    list_updatemenus = []
    nr_region = len(df['Region'])
    data = []
    for i, region in enumerate(np.unique(df['Region'])):
        visible_ = [False] * 2* nr_region
        visible_[2*i:2*i+1] = [True]
        dict_ = {'label': region,
                'method': 'update',
                'args': [{'visible': visible_,
                         'title': 'Neuschnee [cm] pro Jahr in ' + region}]
                }
        list_updatemenus.append(dict_)
        df_ = df[df['Region']==region]
        data.append(go.Scatter(x=df_['Year'],
                               y=df_['Neuschnee'],
                               name=data_name
                   ))
        data.append(go.Scatter(x=df_['Year'],
                               y=df_['Linear Regression Line'],
                               name='Linear Regression' + data_name
                   ))

    layout=go.Layout(title='Neuschnee pro Jahr in verschiedenen Regionen',updatemenus=list([dict(buttons=list_updatemenus)]),
                     barmode='overlay')#defining figure and plotting
    fig = go.Figure(data,layout)
    return fig


In [84]:
df = pd.read_csv('Neuschnee.csv')
np.unique(df['Region'])

array(['Basel-Binningen', 'Bern-Zollikofen', 'Davos', 'Genf-Cointrin',
       'Locarno-Monti', 'Lugano', 'Luzern', 'Neuenburg', 'Samedan',
       'Sitten', 'St. Gallen', 'Säntis', 'Zürich-Fluntern'], dtype=object)

# Neuschnee Davos
Meine Grossmutter geht seit vielen Jahren jährlich zum Skifahren nach Davos. Sie behauptet, dass in den letzten Jahren
weniger Schnee hat. Nach dieser Aussage habe ich ein bisschen nachgedacht und mir ist auch aufgefallen, dass es im letzten
Jahr viele Flächen ohne Schnee auf der Piste hatte. Mit der Neugier im Gepäck, habe ich nach passenden Daten gesucht
und diese auf [Swissopendata.ch](https://opendata.swiss/de/dataset/klimadaten-sonnenscheindauer-niederschlag-temperatur-und-neuschnee2)
gefunden.
Das ist der jährliche Neuschnee in Davos.

In [81]:
df = pd.read_csv('Neuschnee.csv')
fig = plot(df, 'Davos')
fig.show()

Hier habe ich gesehen, dass es so scheint, als gäbe es einen Abwärtstrend. Mit einer Linearen Regressions (Ordinary Least Squares)
lässt sich das auch "wissenschaftlich" veranschaulichen und man sieht, dass es eine Abnahme gibt, von fast einem cm pro Jahr.

In [102]:
df = pd.read_csv('Neuschnee.csv')
df = df[df['Region']=='Davos']
plot_linreg(df).show()

In den anderen Regionen sieht man diese Abnahme ebenfalls. In diesem Plot kann man die einzelnen
Regionen anschauen mit einer Linearen Regression.

In [135]:
df = pd.read_csv('Neuschnee.csv')
selectable_region_plot(df)

False


### Neuschnee pro Region mit Lineare Regression

In [78]:
df = pd.read_csv('Neuschnee.csv')
fig = plot_linreg_subplots(df)
fig.show()

## Jahresniederschlag

In [39]:
df_jahresniederschlag = pd.read_csv('Jahresniederschlag.csv')
fig = plot(df_jahresniederschlag)
fig.show()

## Jahresdurchschnitt Niederschlag für alle Regionen

In [8]:
df_regen_mean = df_jahresniederschlag.groupby('Year', as_index=False).mean()
plot_linreg(df_regen_mean, 'Jahresniederschlag').show()

## Niederschlag pro Region mit Lineare Regression

In [9]:
fig = plot_linreg_subplots(df_jahresniederschlag, 'Jahresniederschlag')
fig.show()

## Temperatur

In [10]:
df_jahrestemperatur = pd.read_csv('Jahrestemperatur.csv')
fig = plot(df_jahrestemperatur)
fig.show()

## Temperaturjahresdurchschnitt für alle Regionen

In [11]:
df_temp_mean = df_jahrestemperatur.groupby('Year', as_index=False).mean()
plot_linreg(df_temp_mean, 'Jahrestemperatur').show()

## Temperatur pro Region mit Lineare Regression

In [12]:
fig = plot_linreg_subplots(df_jahrestemperatur, 'Jahrestemperatur')
fig.show()

## Sonnenscheindauer

In [13]:
df_sonnenschein = pd.read_csv('Sonnenscheindauer.csv')
fig = plot(df_sonnenschein)
fig.show()

## Temperaturjahresdurchschnitt für alle Regionen

In [14]:
df_sonne_mean = df_sonnenschein.groupby('Year', as_index=False).mean()
plot_linreg(df_sonne_mean, 'Sonnenscheindauer').show()

## Temperatur pro Region mit Lineare Regression

In [15]:
fig = plot_linreg_subplots(df_sonnenschein, 'Sonnenscheindauer')
fig.show()

### Korrelation
## Korrelation Regen und Neuschnee

In [16]:
df_jahresniederschlag = pd.read_csv('Jahresniederschlag.csv')
df_neuschnee = pd.read_csv('Neuschnee.csv')
corr_regen_schnee = calculate_corr(df_jahresniederschlag, df_neuschnee)
fig = px.bar(corr_regen_schnee, x='Region', y='Correlation', title='Korrelation Regen und Neuschnee')
fig.show()

x and y must have the same length. with  Luzern


## Korrelation Neuschnee und Temperatur

In [17]:
df_jahrestemperatur = pd.read_csv('Jahrestemperatur.csv')
df_neuschnee = pd.read_csv('Neuschnee.csv')
corr_temp_schnee = calculate_corr(df_jahrestemperatur, df_neuschnee)
fig = px.bar(corr_temp_schnee, x='Region', y='Correlation', title='Korrelation Temperatur und Neuschnee')
fig.show()


x and y must have the same length. with  Luzern
